In [1]:
%load_ext autoreload
%autoreload 2

# Paren Checker

In [2]:

from cases.paren_checker import HighLevelParensBalanceChecker, test_HL_parens_balancer_components, BalancedParensDataset

In [3]:
test_HL_parens_balancer_components()

All Balance tests passed!


True

In [4]:
hl_model = HighLevelParensBalanceChecker()
ll_model = hl_model.get_ll_model()
corr = hl_model.get_correspondence()
dataset = BalancedParensDataset(N_samples=10_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()

making IIT dataset


In [5]:
print(dataset.get_dataset().shape)
print(dataset.get_dataset()[:10]['tokens'])
print(dataset.get_dataset()[:10]['labels'])

(10000, 4)
[[3, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 2], [3, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 2], [3, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2], [3, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 2], [3, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2], [3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2], [3, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 2], [3, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 2], [3, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2], [3, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 2]]
[[0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]]


In [6]:
print(hl_model.get_ll_model_cfg())

HookedTransformerConfig:
{'act_fn': 'relu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 8,
 'd_mlp': 128,
 'd_model': 32,
 'd_vocab': 4,
 'd_vocab_out': 4,
 'default_prepend_bos': True,
 'device': device(type='mps'),
 'dtype': torch.float32,
 'eps': 1e-05,
 'experts_per_token': None,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': True,
 'initializer_range': 0.1414213562373095,
 'load_in_4bit': False,
 'model_name': 'custom',
 'n_ctx': 22,
 'n_devices': 1,
 'n_heads': 4,
 'n_key_value_heads': None,
 'n_layers': 3,
 'n_params': 36864,
 'normalization_type': 'LN',
 'num_experts': None,
 'original_architecture': None,
 'parallel_attn_mlp': False,
 'positional_embedding_type': 'standard',
 'post_embedding_ln': False,
 'rotary_adjacent_pairs': False,
 'rotary_base': 10000,
 'rotary_dim': None,
 'scale_attn_by_i

In [18]:
from iit.model_pairs.strict_iit_model_pair import StrictIITModelPair
from iit.utils.index import Ix
import torch

n_epochs = 1000
training_args = {
    "batch_size": 256,
    "lr": 0.001,
    "num_workers": 0,
    "use_single_loss": False,
    "behavior_weight": 1.,
    "iit_weight": 0., #TODO: Debug IIT loss.
    "strict_weight": 0.5,
    "clip_grad_norm": 1.0,
    "iit_weight_schedule" : lambda s, i: s,
    "strict_weight_schedule" : lambda s, i: s,
    "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
    "early_stop" : True,
    "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
    "scheduler_kwargs": dict(start_factor=1, end_factor=0, total_iters=n_epochs),
    "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
    "scheduler_mode": "max", #for ReduceLRonPlateau
}

class LastTokenStrictIITModelPair(StrictIITModelPair):
    def get_label_idxs(self):
        return Ix[[None,-1]]

model_pair = LastTokenStrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)

In [19]:
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 1000}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x3496e1800>, 'strict_weight_schedule': <function <lambda> at 0x3496e0860>, 'behavior_weight_schedule': <function <lambda> at 0x3496e11c0>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 0.0, 'behavior_weight': 1.0, 'strict_weight': 0.5}
Epoch 1: lr: 9.99e-04, iit_weight: 0.00e+00, behavior_weight: 1.00e+00, strict_weight: 5.00e-01, train/iit_loss: 0.00e+00, train/behavior_loss: 8.50e-02, train/strict_loss: 1.61e-02, val/iit_loss: -6.44e+04, val/IIA: 84.10, val/accuracy: 83.90, val/strict_accuracy: 99.90
Epoch 2: lr: 9.98e-04, iit_weight: 0.00e+00, behavior_weight: 1.00e+00,

KeyboardInterrupt: 